# ЛАБОРАТОРИЯ

# Часть II. Первый подход к прогнозированию

### Оглавление

[Библиотеки и утилиты](#Библиотеки-и-утилиты)

[Разбор сырых данных](#Разбор-сырых-данных)

[Предварительная обработка и анализ](#Предварительная-обработка-и-анализ)

[Визуальный анализ данных](#Визуальный-анализ-данных)

[Один ряд как пример](#Один-ряд-как-пример)

[Немного новых признаков](#Немного-новых-признаков)

[Base-line модель: подготовка данных](#Base-line-модель:-подготовка-данных)

[Base-line модель: обучение](#Base-line-модель:-обучение)

[ОПЦИОНАЛЬНО: тюнинг base-line модели](#ОПЦИОНАЛЬНО:-тюнинг-base-line-модели)

[САМОСТОЯТЕЛЬНО](#САМОСТОЯТЕЛЬНО)

### Библиотеки и утилиты

In [ ]:
import os
import json
import pickle
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from datetime import datetime
from sklearn.utils import shuffle
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM, TimeDistributed
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from hyperopt import hp, tpe, space_eval
from hyperopt.fmin import fmin
pd.set_option('display.max_columns', None)
print('tensorflow version:', tf.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    for gpu_device in gpu_devices:
        print('device available:', gpu_device)

In [ ]:
MODEL_PATH = './models'
if not os.path.exists(MODEL_PATH):
    os.mkdir(MODEL_PATH)

def set_all_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_all_seeds(2035)

### Визуальный анализ данных

In [ ]:
df = pd.read_csv('cpt_power_data.csv', sep='\t', encoding='utf-8', index_col=0)
df['timestamp_value'] = pd.to_datetime(df['timestamp_value'])
df.head()

In [ ]:
for col in df:
    print(f'{col}:', len(df[col].unique()), '| sample:', df[col].unique()[:3])

In [ ]:
plt.figure(figsize=(16, 6))
for ch_serial in df['measuringpoint_serial'].unique():
    plt.plot(df[df['measuringpoint_serial'] == ch_serial].timestamp_value, 
             df[df['measuringpoint_serial'] == ch_serial].value_text, 
             label=ch_serial)
plt.legend()
plt.show()

### Один ряд как пример

In [ ]:
ch_serial = df['measuringpoint_serial'].unique()[0]
channel = pd.DataFrame(
    data=list(df[df['measuringpoint_serial'] == ch_serial].value_text),
    index=list(df[df['measuringpoint_serial'] == ch_serial].timestamp_value)
)
channel.columns = ['pwr']
print(channel.shape)
channel.head()

In [ ]:
plt.figure(figsize=(16, 4))
plt.plot(channel)
plt.title(f'Channel No {ch_serial}')
plt.show()

### Немного новых признаков

Мануал по datetime, полезно для признаков на основе дат https://docs.python.org/3/library/datetime.html#datetime.datetime.weekday

In [ ]:
channel['n_day'] = channel.index.day.astype(np.int8)
channel['n_week'] = channel.index.week.astype(np.int8)
channel['n_month'] = channel.index.month.astype(np.int8)
channel['w_day'] = channel.index.weekday.astype(np.int8)
channel['is_weekend'] = (channel['w_day'] >= 5).astype(np.int8)
channel.head()

In [ ]:
fig, ax1 = plt.subplots(figsize=(16, 6))

ax1.set_xlabel('time')
ax1.set_ylabel('power')
ax1.plot(channel.index, channel.pwr, label='power', color='grey')
ax1.tick_params(axis='y')

ax2 = ax1.twinx()
ax2.set_ylabel('days')
for col in [x for x in channel.columns if '_' in x]:
    ax2.plot(channel.index, channel[col], label=col)
ax2.tick_params(axis='y')

fig.legend()
plt.title('Channel: power and time features')
plt.show()

### Base-line модель: подготовка данных

Полезная статья по LSTM для прогнозирования временных рядов https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/

Статья про аналогичную задачу https://machinelearningmastery.com/how-to-develop-lstm-models-for-multi-step-time-series-forecasting-of-household-power-consumption/

In [ ]:
scaler, scaler_pwr = MinMaxScaler(feature_range=(0, 1)), MinMaxScaler(feature_range=(0, 1))
scaler_pwr.fit(channel['pwr'].values.reshape(-1, 1))
with open(f'{MODEL_PATH}/scaler_pwr.pkl', 'wb') as file:
    pickle.dump(scaler_pwr, file)
channel = scaler.fit_transform(channel)
print('total elements:', len(channel))
print('one element of channel:', channel[0])

In [ ]:
def get_dataset(series, col_look, look_back, look_fwd, cols_features):
    X, y = [], []
    for i in range(len(series[:, col_look]) - look_back - look_fwd):
        temp_X = []
        temp_X.append(series[:, col_look][i:(i + look_back)])
        for col in cols_features:
            temp_X.append(series[:, col][(i + look_fwd):(i + look_back + look_fwd)])
        X.append(temp_X)
        y.append(series[:, col_look][(i + look_back):(i + look_back + look_fwd)])
    return np.array(X), np.array(y)

In [ ]:
look_fwd = 7 * 24 * 2 # days * hours * half an hour
print('look forward:', look_fwd)
look_back = 2 * look_fwd 
print('look back:', look_back)
X, y = get_dataset(channel, 0, look_back, look_fwd, [1, 2, 3, 4, 5])
print('X shape:', X.shape, '| y shape:', y.shape)
print('train sample:', X[0][:, 0])

In [ ]:
def get_train_test(X, y, test_size=.25):
    cut = int((1 - test_size) * len(y))
    X_train, y_train = X[:cut], y[:cut]
    X_test, y_test = X[cut:], y[cut:]
    
    # LSTM feed [samples, time steps, features]
    print('as is:')
    print('\tX train shape:', X_train.shape, '| X test shape:', X_test.shape)
    print('\ty train shape:', y_train.shape, '| y test shape:', y_test.shape)
    print('\nreshaped to LSTM pattern [samples, time steps, features]:')
    X_train = np.array([x.T for x in X_train])
    X_test = np.array([x.T for x in X_test])
    print('\tX train shape:', X_train.shape, '| X test shape:', X_test.shape)
    print('\ty train shape:', y_train.shape, '| y test shape:', y_test.shape)
    
    return X_train, y_train, X_test, y_test

In [ ]:
X_train, y_train, X_test, y_test = get_train_test(X, y, test_size=.25)

### Base-line модель: обучение

In [ ]:
def get_model(units, n_features, dropout,
              look_back, look_fwd,
              stack=False, loss='mse'):
    model = Sequential()
    if stack:
        model.add(LSTM(units=units, 
                       input_shape=(look_back, n_features), 
                       return_sequences=True, 
                       dropout=dropout, 
                       recurrent_dropout=dropout))
        model.add(LSTM(units=units,
                       dropout=dropout, 
                       recurrent_dropout=dropout))
    else:
        model.add(LSTM(units=units, 
                       input_shape=(look_back, n_features),
                       dropout=dropout, 
                       recurrent_dropout=dropout))
    model.add(Dense(look_fwd))
    model.add(Activation('linear'))
    adam = optimizers.Adam(lr=.001, clipvalue=.5, clipnorm=1)
    model.compile(loss=loss, optimizer=adam)
    return model

In [ ]:
model = get_model(
    units=256, 
    n_features=6, 
    dropout=.4,
    look_back=look_back, 
    look_fwd=look_fwd,
    stack=False, 
    loss='mse'
)
model.summary()

In [ ]:
%%time
checkpoint_path = f'{MODEL_PATH}/model.hdf5'
earlystopper = EarlyStopping(
    monitor='val_loss', 
    patience=40, 
    verbose=1,
    mode='min'
)
lrreducer = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=.1, 
    patience=20, 
    verbose=1, 
    min_lr=1e-6,
    mode='min'
)
checkpointer = ModelCheckpoint(
    checkpoint_path, 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=True,
    save_weights_only=True, 
    mode='min'
)
callbacks = [earlystopper, checkpointer, lrreducer]
history = model.fit(
    X_train, y_train, 
    epochs=1000, 
    batch_size=128, 
    validation_data=(X_test, y_test), 
    verbose=1,
    callbacks=callbacks,
    shuffle=False
)

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16, 4))
y_pred = model.predict(X_test[0:1])[0]
plt.plot(y_test[0], label='true')
plt.plot(y_pred, label='predict')
plt.legend()
plt.show()

In [ ]:
y_pred = model.predict(X_test[0:1])[0]

with open(f'{MODEL_PATH}/scaler_pwr.pkl', 'rb') as f:
    scaler_pwr = pickle.load(f)

plt.figure(figsize=(16, 4))
past = range(len(X_train[0][:, 0]))
future = range(len(X_train[0][:, 0]), len(X_train[0][:, 0]) + len(y_test[0]))
plt.plot(past, 
         scaler_pwr.inverse_transform(X_train[-1][:, 0].reshape(-1, 1)), 
         label='train')
plt.plot(future, 
         scaler_pwr.inverse_transform(y_test[0].reshape(-1, 1)), 
         label='true')
plt.plot(future, 
         scaler_pwr.inverse_transform(y_pred.reshape(-1, 1)), 
         label='predict')
plt.legend()
plt.show()

In [ ]:
channel_ = pd.DataFrame(
    data=list(df[df['measuringpoint_serial'] == ch_serial].value_text),
    index=list(df[df['measuringpoint_serial'] == ch_serial].timestamp_value)
)
plt.figure(figsize=(16, 4))
start_point = len(X_train) + look_back
plt.plot(channel_[(start_point - 2 * look_back):(start_point + len(y_pred))], 
         label='true')
plt.plot(channel_.index[start_point:(start_point + len(y_pred))],
         scaler_pwr.inverse_transform(y_pred.reshape(-1, 1)), 
         label='predict')
plt.legend()
plt.show()

### ОПЦИОНАЛЬНО: тюнинг base-line модели

Мануал по hyperopt https://hyperopt.github.io/hyperopt/

In [ ]:
def train_model(X_train, y_train, X_test, y_test,
                units, n_features, dropout, stack,
                loss='mae', patience=40, batch_size=128):
    model = get_model(
        units=units, 
        n_features=n_features, 
        dropout=dropout,
        look_back=look_back, 
        look_fwd=look_fwd,
        stack=stack, 
        loss=loss
    )
    checkpoint_path = f'{MODEL_PATH}/model.hdf5'
    earlystopper = EarlyStopping(
            monitor='val_loss', 
            patience=patience, 
            verbose=0,
            mode='min'
    )
    lrreducer = ReduceLROnPlateau(
        monitor='val_loss', 
        factor=.1, 
        patience=int(patience / 2), 
        verbose=0, 
        min_lr=1e-6,
        mode='min'
    )
    checkpointer = ModelCheckpoint(
        checkpoint_path, 
        monitor='val_loss', 
        verbose=0, 
        save_best_only=True,
        save_weights_only=True, 
        mode='min'
    )
    callbacks = [earlystopper, checkpointer, lrreducer]
    history = model.fit(
        X_train, y_train, 
        epochs=1000, 
        batch_size=batch_size, 
        validation_data=(X_test, y_test), 
        verbose=2,
        callbacks=callbacks,
        shuffle=False
    )
    return history, model

In [ ]:
PARAMS = {
    'units': 128,
    'dropout': .4,
    'stack': False
}
space = {
    'units': hp.choice('units', [64, 128, 256]),
    'dropout': hp.quniform('dropout', 0, .5, .01),
    'stack': hp.choice('stack', [True, False])
}
MAX_EVALS = 5

In [ ]:
PARAMS_OPT = PARAMS.copy()
best_model = None
best_metric = 10e8

def objective(params):
    print('=' * 50, '\n', '=' * 50)
    global PARAMS_OPT, best_model, best_metric
    PARAMS_OPT.update(params)
    history, model = train_model(
        X_train, y_train, X_test, y_test,
        units=PARAMS_OPT['units'], 
        n_features=6, 
        dropout=PARAMS_OPT['dropout'], 
        stack=PARAMS_OPT['stack'],
        loss='mae', 
        patience=10,
        batch_size=128
    )
    metric = min(history.history['val_loss'])
    if metric < best_metric:
        best_metric = metric
        best_model = model
    print(f'\nparams: {params} | metric: {metric}\n')
    return metric

best_hopt = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=MAX_EVALS)
print('best search:', best_hopt, '\nbest params:', space_eval(space, best_hopt))
PARAMS_OPT.update(space_eval(space, best_hopt))

params_file = f'{MODEL_PATH}/hopt_params.json'
with open(params_file, 'w') as file:
    json.dump(PARAMS_OPT, file)

In [ ]:
plt.figure(figsize=(16, 4))
y_pred = best_model.predict(X_test[0:1])[0]
plt.plot(y_test[0], label='true')
plt.plot(y_pred, label='predict')
plt.title(f'Best model predictions, best metric={best_metric:.4f}')
plt.legend()
plt.show()

### САМОСТОЯТЕЛЬНО

#### 1. Прогноз на две недели для следующих рядов

Предлагается изучить два ряда с номерами каналов [11202786, 13150883]:

In [ ]:
plt.figure(figsize=(16, 6))
for ch_serial in [11202786, 13150883]:
    plt.plot(df[df['measuringpoint_serial'] == ch_serial].timestamp_value, 
             df[df['measuringpoint_serial'] == ch_serial].value_text, 
             label=ch_serial)
plt.legend()
plt.show()

#### 2. Эксперименты с дополнительными признаками

Можно учесть:
1. Фактор праздников (первая неделя мая)
2. Погодные условия (температура)
3. Фазы луны (ну а вдруг...)

#### 3. Тюнинг параметров

Можно тюнить почти все:
1. Размер батча
2. Patience
3. Learning rate
4. Optimizer
5. Архитектура сети (количество слоев, функция активации, GRU vs LSTM)

#### 4. Оценка решения

По метрике __mean_squared_error__